# Basic Ent Creator

This is a basic workflow to stand up an **Ent**: a live digital twin of an agroecology/agroforestry project that helps you understand the state of your land and make decisions about it. It is a "basic" workflow because it allows you to customize and fine-tune an existing Ent model to your project, but does not allow you to modify the model or create a new one.

The capabilities that this workflow implements:

1. Interact with the Fangorn ontology to ensure that the crop/plant species, data sources, agricultural practices, outcomes of interest and external factors that are relevant for your project are represented.
1. Create a project configuration file that represents your project.
1. Add parameter tables for effect sizes relative to new entries in the ontology.
1. Select the agent model that fits your project.
1. Spin up an Ent and attach it to your project's private data assets.
1. Generate an assessment and interpret its results and recommendations.
1. Connect your Ent to the Fangorn network to get streaming parameter updates from peer nodes.

This workflow does not support modifying the internals of the model, that is, changing the functional form of the relationships between variables, nor adding new variables. We are working on an advanced workflow to help you do that. Meanwhile, for a thorough introduction to Ents and Fangorn for modelers, see the `Get Started` and `Indoor Agriculture Modeler's Lab` notebooks.

## How to use this notebook

We have created a ficticious project called Sierra, inspired by the [story](https://www.theatlantic.com/science/archive/2018/08/amaizeballs/567140/) of the discovery of self-fertilizing corn in Mexico's Sierra Mixe. You use this notebook by replacing the ficticious inputs below with your project's real ones, and running it.

Throughout the notebook, user inputs are represented as `CONSTANTS IN ALL CAPS` with comments where you should replace your own values for your project. These indicated like so:

```python
# TODO(user): REPLACE
INPUT = "foo"
```

You can generate the values manually, by following the examples given, or use `Entish`, our handy language AI, to automatically generate values in the right structured format. **NOTE**: As a language model, `Entish` does not always get things right. You are responsible for checking its outputs!

We recommend editing and running through the whole notebook in order and making sure that you remove the `TODO`s as you go.

In [ ]:
autogen = Entish.gen_data("""
Example outdoor farm called Sierra, for Digital Gaia. It grows maize in two separate plots: \
plot 1 has a hybrid dent corn, using  and plot 2 has a local heirloom variety called Sierra Heirloom. \
Plot 1 uses a typical amount of fertilizer, while plot 2 just uses minimal amounts. \
We're interested in analyzing the yield, fertilizer amount used, soil nitrogen content, and soil organic matter content. \
We have access to self-reported annual data on yield and fertilizer use, from the farmer herself (fatima@sierra.example). \
A local laboratory called SierraLab (lab@sierralab.example) has done soil tests for the last 5 growing seasons, \
which include the SOM content and nitrogen content. \
""")
print(autogen)

In [ ]:
# TODO(user): REPLACE
PROJECT_NAME = "Example.DigitalGaia.Sierra"

## Your project and the Fangorn ontology

Fangorn is a symbolic system: variables typically have interpretations in terms of concepts in an **ontology**, which are either hardcoded by the modeler or discovered by an algorithm. Fangorn assumes that all Ents use a common (global) ontology, which enables them to communicate parameters (ex: effect sizes) with each other. An Ent may have an ontology that disagrees with the global one; however, that will effectively mean that it's partially unable to communicate about those concepts with the rest of the network.

First we will declare the local ontology for our Ent. The `LocalOntology` function maps from the names we give it to the known global names, using a set of heuristics, and generates an ontology dictionary including any global names it found and any 'local-only' names that it couldn't find.

Later we will see how to request an update to add these "missing" names to the global ontology.

In [ ]:
#TODO(user): REPLACE
LOCAL_ONTOLOGY = autogen.LOCAL_ONTOLOGY
LOCAL_ONTOLOGY = {
    "species":      ["Maize.HybridDentCorn", "Maize.SierraHeirloom"],
    "actions":      ["PlantSeeds" ,"HarvestCrops", "FertilizeSoil.Standard", "FertilizeSoil.Minimal", "IrrigateCrops", "Outdoor"],
    "observables":  ["Yield", "FertilizerUse", "SoilOrganicMatter", "SoilNitrogen", "NDVI"],
    "outcomes":     ["Yield", "FertilizerUse", "SoilOrganicMatter", "SoilNitrogen"]
}
ontology = LocalOntology(LOCAL_ONTOLOGY)

## Project configuration

The project configuration file tells the Ent everything it needs to know in order to run and to connect to the network:
* The project's name, which is expected to be a unique identifier within each network.
* The project's starting date and (optionally) its duration.
* The geographical boundaries of each **lot** within the project. Lot names are arbitrary.
* The **strategies** chosen for each lot in the project. Strategies are the main factor in determining which model the Ent selects, as they correspond to a set of interventions and plant species that must be modeled, as well as their **objectives** or success metrics. Strategy names are also arbitrary.

The below uses a couple of utility functions to generate repetitive, correctly formatted data:
* `ObjectiveFunction` generates a scalar-valued objective per cycle, given an outcome variable. Optionally you can pass a threshold, whether to count the outcome as "more is better" or "less is better", and an aggregation function.
* `AgPolicy` generates a sequence of actions for the agricultural cycle. Note that these are priors, and the Ent will attempt to infer what actual actions were taken when, based on observations.

In [ ]:
#TODO(user): REPLACE all of the below
START_DATE = "2018-03-31"  # Date of first planting
LOOKBEHIND = 0  # Number of weeks before START_DATE to analyze
CYCLE_LENGTH = 52  # Number of weeks between plantings
SEASON_LENGTH = 22  # Number of weeks between planting and harvest
PLANTING_DURATION = 1
HARVEST_DURATION = 1
FERTILIZER_INTERVAL = 6

PROJECT_CONFIG = {
	"name": PROJECT_NAME,
	"start_date": START_DATE,
	"lots": [{
		"name": "Lot 1",
        "bounds": {
            "type": "Polygon",
            "coordinates": [[
                [-95.78718140290917, 17.16661692985312],
                [-95.78718140290917, 17.165812257398287],
                [-95.78611758414853, 17.165812257398287],
                [-95.78611758414853, 17.16661692985312],
                [-95.78718140290917, 17.16661692985312]
            ]]
        },
        "strategy": "standard-corn"
	}, {
		"name": "Lot 2",
        "bounds": {
            "type": "Polygon",
            "coordinates": [[
                [-95.785641081995, 17.1665851664995],
                [-95.785641081995, 17.165780494044668],
                [-95.78457726323435, 17.165780494044668],
                [-95.78457726323435, 17.1665851664995],
                [-95.785641081995, 17.1665851664995]
            ]]
        },
        "strategy": "heirloom-low-fertilizer"
	}],
	"strategies": [{
        "name": "standard-corn",
        "species": [ ontology["species"]["Maize.HybridDentCorn"] ],
        "interventions": ontology["actions"],
        "objectives": [ObjectiveFunction(ontology["outcomes"]["Yield"]),
                       ObjectiveFunction(ontology["outcomes"]["FertilizerUse"], positive=False),
                       ObjectiveFunction(ontology["outcomes"]["SoilNitrogen"]),
                       ObjectiveFunction(ontology["outcomes"]["SoilOrganicMatter"])],
        "policy": AgPolicy(cycle_length=52,
                           cycle_start=START_DATE,
                           cycle_length=CYCLE_LENGTH,
                           season_length=SEASON_LENGTH,
                           fertilizer_action=ontology["actions"]["FertilizeSoil.Standard"],
                           fertilizer_interval=FERTILIZER_INTERVAL,
                           outdoor=True)
    }, {
        "name": "heirloom",
        "species": [ ontology["species"]["Maize.SierraHeirloom"] ],
        "interventions": ontology["actions"],
        "objectives": [ObjectiveFunction(ontology["outcomes"]["Yield"]),
                       ObjectiveFunction(ontology["outcomes"]["FertilizerUse"], positive=False),
                       ObjectiveFunction(ontology["outcomes"]["SoilNitrogen"]),
                       ObjectiveFunction(ontology["outcomes"]["SoilOrganicMatter"])],
        "policy": AgPolicy(cycle_length=52,
                           cycle_start=START_DATE,
                           cycle_length=CYCLE_LENGTH,
                           season_length=SEASON_LENGTH,
                           fertilizer_action=ontology["actions"]["FertilizeSoil.Minimal"],
                           fertilizer_interval=FERTILIZER_INTERVAL,
                           outdoor=True)
    }]
}

## Initializing the Ent

Now we take the above configurations and create an Ent object. The constructor selects the model from the Fangorn library that best matches the configurations.

Note that our local ontology includes some concepts that aren't in the Fangorn global ontology:
* It refines `Maize` into `Maize.HybridDentCorn` and `Maize.SierraHeirloom`.
* Similarly, it refines `FertilizeSoil` into `FertilizeSoil.Standard` and `FertilizeSoil.Minimal`.

Not only does it select the best "generic" model for this project, the Ent also refines it by adding "features" as needed. What does this mean?

Spoiler alert: it turns out that Lot 2 requires significantly less fertilizer than Lot 1 to produce the same yield. The Ent will be able to correctly infer posteriors for the generic parameter (effect size of `FertilizeSoil` on `Yield.Maize`) at the lot level. However, it will *also* be able to infer the specific effect size of `FertilizeSoil.Minimal` on `Yield.Maize.SierraHeirloom`!

However, the Ent can only share the generic parameter with the world, as the other Ents in the network do not know about those refined concepts...

**YET!**

What if the local ontology includes concepts that aren't refinements, but completely new? The Ent would just ignore those concepts.

In [ ]:
ent = Ent(PROJECT_CONFIG)

## Starting the Ent

* `report_sources`: A list of sources to fetch reports from. Currently the only supported source type is a directory on a mounted file system. Coming soon: support for streaming APIs.
* `long_lived`: If true, after reading the available reports, the Ent will continue running and periodically fetch for new reports to update assessments. Coming soon: Docker image. 

## Adding concepts to the global ontology

So how does our Ent tell the world about the benefits of the Sierra heirloom maize? Well, it turns out that, when LocalOntology ruan, it already edited/modified the files you needed to create the ontology change request. The below commands easily generate a commit and pull request.

Once the concepts are added, the owners of other Ents get notified and can restart their Ents with the new, refined ontology and model. This entails a process called revisitation: The Ents will re-process every report they got, in order to see if there's any information referring to the new concepts that they previously missed because it wasn't in their ontology.

## Attaching the Ent to a network

Note the `NETWORK` input below:
* If you leave it as None, your Ent will run standalone.
* If you set it to `testnet`, it will run in our test network. Running Ents in the testnet is very useful for testing and validation.
* If you set it to `mainnet`, it will run in our production network and will attempt to generate "official" credentials from Fangorn for its assessments. You should only use this once you have thoroughly validated your project's configurations and private data sources, as running a "bad" Ent on Fangorn mainnet will cause your user and project trust scores to be downgraded, which will be visible on your assessments and will reduce any monetary compensations due to you.

In [ ]:
# TODO(user): REPLACE
NETWORK = None
ent.attach(NETWORK)